# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# path
cities_path="../WeatherPy/Output/cities_weather.csv"
# Read the cities data
cities_data= pd.read_csv(cities_path)
cities_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cidreira,-30.1811,-50.2056,20.86,71,52,7.62,BR,2021-02-04 23:05:41
1,Fortuna,40.5982,-124.1573,12.78,71,1,5.14,US,2021-02-04 23:05:41
2,Port Elizabeth,-33.9180,25.5701,20.00,88,0,2.06,ZA,2021-02-04 23:05:41
3,Ayutla,20.1167,-104.3333,29.48,15,0,3.99,MX,2021-02-04 23:01:34
4,Puerto Ayora,-0.7393,-90.3518,30.00,62,0,5.66,EC,2021-02-04 23:04:03
...,...,...,...,...,...,...,...,...,...
567,Bahía Blanca,-38.7196,-62.2724,26.60,27,0,4.76,AR,2021-02-04 23:07:12
568,Acapulco de Juárez,16.8634,-99.8901,29.00,70,1,2.57,MX,2021-02-04 23:02:53
569,Tipitapa,12.1973,-86.0971,31.00,45,20,4.12,NI,2021-02-04 23:07:12
570,Chachapoyas,-6.2317,-77.8690,15.54,87,97,1.29,PE,2021-02-04 23:07:12


In [3]:
cities_data.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = cities_data[["Lat", "Lng"]]
# Fill NaN values and convert to float
humidity = cities_data["Humidity"].astype(float)

In [7]:
# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
cities_ideal_data=cities_data.loc[cities_data['Wind Speed']<=10]
#cities_data['Cloudiness']==0 & cities_data['Max Temp']<80 & cities_data['Max Temp']>60 &  ]
cities_ideal_data

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Add columns for Country name, Hotel name, hotel rating
cities_ideal_data["Country"] = ""
cities_ideal_data["Hotel name"] = ""
cities_ideal_data["Hotel raiting"] = ""
cities_ideal_data["Hotel Address"] = ""

cities_ideal_data.head()

In [ ]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in cities_ideal_data.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        cities_ideal_data.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        cities_ideal_data.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        cities_ideal_data.loc[index, "Country"] = name_address["results"][0]["rating"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
